In [1]:
from keras.models import Model
from keras.layers import Flatten, Dense, Input
from keras_vggface.vggface import VGGFace
from keras import backend as K
from keras_vggface import utils

#custom parameters
hidden_dim = 512
image_size = 224
train_dir = "6"
validation_dir = "6_valid"
classes = 2
EPOCHS = 30

vgg_model = VGGFace(include_top=False, input_shape=(224, 224, 3))
last_layer = vgg_model.get_layer('pool5').output
x = Flatten(name='flatten')(last_layer)
x = Dense(hidden_dim, activation='relu', name='fc6')(x)
x = Dense(hidden_dim, activation='relu', name='fc7')(x)
out = Dense(classes, activation='softmax', name='fc8')(x)
custom_vgg_model = Model(vgg_model.input, out)

layer_count = 0
for layer in custom_vgg_model.layers:
    layer_count = layer_count+1
for l in range(layer_count-3):
    custom_vgg_model.layers[l].trainable=False
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [2]:
from keras.preprocessing.image import ImageDataGenerator 

train_datagen = ImageDataGenerator(
      preprocessing_function=utils.preprocess_input,
      rotation_range = 10,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range = 0.2,
      fill_mode = "reflect",
      horizontal_flip=True,)
 
validation_datagen = ImageDataGenerator(preprocessing_function=utils.preprocess_input)
 
# Change the batchsize according to your system RAM
train_batchsize = 20
val_batchsize = 30
 
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_size, image_size),
        color_mode = 'rgb',
        batch_size=train_batchsize,
        class_mode="categorical")
 
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(image_size, image_size),
        color_mode = 'rgb',
        batch_size=val_batchsize,
        class_mode="categorical",
        shuffle=False)

Found 5735 images belonging to 2 classes.
Found 1552 images belonging to 2 classes.


In [3]:
import csv
import os
import cv2
from keras.preprocessing.image import img_to_array
import numpy as np

def readFaces(dir,mydict):
    faces = []
    ages = []
    directory = os.fsencode(dir)
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".png") or filename.endswith(".jpg"): 
            path = ".\\" + dir + "\\" + filename
            img = cv2.imread(path)
            img = cv2.resize(img, (224, 224))
            img = img.astype("float16")
            img = img_to_array(img)
            img = utils.preprocess_input(x=img,version=1)
            img = np.expand_dims(img, axis=0)
            faces.append(img)
            ages.append(mydict.get(filename))
    return np.asarray(faces),ages

with open('ground_truth/test_gt.csv', 'r') as f:
    reader = csv.reader(f)
    mydict = dict((rows[0],rows[1]) for rows in reader)

x_test,y_test = readFaces("test_deep",mydict)

def arrangeLabels(Y,bins,labels):
    returnLabels = []
    for label in Y:
        for index in range(len(bins)):
            if float(label) > bins[index]:
                continue;
            else:
                returnLabels.append(labels[index-1])
                break;
                
    return returnLabels

bins = [0.0, 48.0,np.inf]
labels = [0, 1]

y_test = np.array(arrangeLabels(y_test,bins,labels))

In [4]:
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
custom_vgg_model.compile(optimizer=SGD(lr=0.000001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
history = custom_vgg_model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=EPOCHS,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)

Epoch 1/30
287/286 [==============================] - 120s 419ms/step - loss: 1.2674 - acc: 0.8613 - val_loss: 1.2181 - val_acc: 0.8789
Epoch 2/30
287/286 [==============================] - 113s 394ms/step - loss: 0.8923 - acc: 0.8912 - val_loss: 0.8830 - val_acc: 0.8930
Epoch 3/30
287/286 [==============================] - 113s 394ms/step - loss: 0.7183 - acc: 0.9057 - val_loss: 0.7464 - val_acc: 0.8956
Epoch 4/30
287/286 [==============================] - 112s 391ms/step - loss: 0.6100 - acc: 0.9124 - val_loss: 0.6719 - val_acc: 0.9091
Epoch 5/30
287/286 [==============================] - 113s 393ms/step - loss: 0.5303 - acc: 0.9196 - val_loss: 0.6467 - val_acc: 0.9098
Epoch 6/30
287/286 [==============================] - 112s 390ms/step - loss: 0.4854 - acc: 0.9231 - val_loss: 0.5916 - val_acc: 0.9188
Epoch 7/30
287/286 [==============================] - 113s 393ms/step - loss: 0.4524 - acc: 0.9314 - val_loss: 0.5709 - val_acc: 0.9207
Epoch 8/30
287/286 [============================

In [5]:
import matplotlib.pyplot as plt
import numpy as np

# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), history.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), history.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), history.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper left")
plt.savefig("Ranking6")

In [6]:
confusion_matrix = [[0 for x in range(2)] for y in range(2)] # Initialing confusion_matrix
print("[INFO] loading network...")
hit = 0
total = len(x_test)
for index in range(total):
    # classify the input image
    print("[INFO] classifying image...")
    proba = custom_vgg_model.predict(x_test[index])[0]
    idx = np.argmax(proba)
    confusion_matrix[idx][y_test[index]]+=1
    print("Predicted "+str(idx)," Actual "+str(y_test[index]))
    hit += idx==y_test[index]

print(np.asmatrix(confusion_matrix))
print(hit/total)

[INFO] loading network...
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 1  Actual 1
[INFO] classifying image...
Predicted 0  Actual 1
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 1
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 1
[INFO] classifying image...
Predicted 0  Actual 1
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 1  Actual 1
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image

Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 1
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 1
[INFO] classifying image...
Predicted 1  Actual 1
[INFO] classifying image...
Predicted 1  Actual 0
[INFO] classifying image...
Predicted 1  Actual 1
[INFO] classifying image...
Predicted 1  Actual 1
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...


Predicted 1  Actual 1
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 1  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 1  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 1  Actual 1
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...


Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 1
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 1
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 1  Actual 1
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...


Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 1  Actual 1
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 1  Actual 1
[INFO] classifying image...
Predicted 0  Actual 1
[INFO] classifying image...
Predicted 1  Actual 1
[INFO] classifying image...
Predicted 1  Actual 1
[INFO] classifying image...


...

Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 1
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 1  Actual 1
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 1  Actual 1
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...


Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 1
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...


Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 1
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 1  Actual 1
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 1
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 1
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 1  Actual 1
[INFO] classifying image...


Predicted 0  Actual 0
[INFO] classifying image...
Predicted 1  Actual 1
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 1  Actual 1
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 1
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 1  Actual 1
[INFO] classifying image...
Predicted 0  Actual 1
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 1  Actual 1
[INFO] classifying image...


Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 1  Actual 0
[INFO] classifying image...
Predicted 1  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[[1449  107]
 [  66  242]]
0.9071888412017167


In [7]:
custom_vgg_model.save("Ranking6")

In [8]:
img = cv2.imread("hatice.png")
img = cv2.resize(img, (224, 224))
img = img.astype("float16")
img = utils.preprocess_input(img)
img = img_to_array(img)
img = np.expand_dims(img, axis=0)
print(img.shape)
proba = custom_vgg_model.predict(img)[0]
idx = np.argmax(proba)
print("Predicted "+str(idx))

(1, 224, 224, 3)
Predicted 0
